In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os, collections
import numpy as np
import pandas as pd
import scipy, soundfile, librosa
import mir_eval
from time import time
from fnmatch import fnmatch
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from thundersvm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer  # for imputation

import warnings
warnings.filterwarnings('ignore')

# Load data

In [2]:
data_root = '/import/c4dm-datasets/ChickCallDataset/annotated/EUSIPCO22/'
wav_files = []  
with open('file_names_callType.txt', 'r') as f:
    Lines = f.readlines()
    for line in Lines: 
        wav_files.append(line.strip())
        
print('Number of audio files:', format(len(wav_files)))
print(*wav_files, sep='\n')

# chick call annotation files
csv_files = [file.replace('.wav', '.csv') for file in wav_files]
chickID = [int(wav_files[k][:2]) for k in range(len(wav_files))]
print('Chick ID:', *chickID, sep=' ')

Number of audio files: 4
85SM_2020-01-27_14-40-03.wav
87SM_2020-01-27_15-43-59.wav
89SF_2020-01-27_16-51-22.wav
91SM_2020-01-30_10-20-53.wav
Chick ID: 85 87 89 91


In [3]:
# get call segs information
anno = []
for file in csv_files:
    file_anno = pd.read_csv(data_root+file)
    file_anno['chick'] = [int(file[:2])] * len(file_anno)
    anno.append(file_anno)
    
anno = pd.concat(anno, axis=0, ignore_index=True)

call_segs = pd.DataFrame({'chickID':[], 'callID':[], 'truth_start': [], 'truth_end': [], 'truth_label':[]})
call_segs['chickID'] = list(anno['chick'][1::2])
call_segs['callID'] = np.arange(len(call_segs['chickID']), dtype=int)
call_segs['truth_start'] = list(anno['time'][0::2])
call_segs['truth_end'] = list(anno['time'][1::2])
call_segs['truth_label'] = list(anno['anno'][1::2])

call_segs.loc[(call_segs.truth_label == 'p'),'truth_label']= 1
call_segs.loc[(call_segs.truth_label == 'd'),'truth_label']= 2
call_segs.loc[(call_segs.truth_label == 'q'),'truth_label']= 3
call_segs

,chickID,callID,truth_start,truth_end,truth_label
0,85,0,0.000000,0.123356,1
1,85,1,0.336961,0.595465,2
2,85,2,0.830385,1.137415,2
3,85,3,1.258957,1.551020,2
4,85,4,1.659864,1.991837,2
...,...,...,...,...,...
3008,91,3008,629.097506,629.471202,2
3009,91,3009,632.406349,632.756825,2
3010,91,3010,632.883809,633.269841,2
3011,91,3011,633.361270,633.747302,2


# MFCC feature extraction

In [4]:
sr = 44100
hop_sample= 1103  # 25ms
print('frame size: %sms' % (int(hop_sample/44100*1000)))

# MFCC feature extraction, default: n_fft=2048, hop_length=512 
feature = {k:[] for k in range(len(wav_files))}

print('MFCC feature shape: ')
for k in range(len(wav_files)):
    y, sr = soundfile.read(data_root + wav_files[k])
    y = np.mean(y,1)
    feature[k] = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=24, hop_length=hop_sample)
    print(feature[k].shape)

frame size: 25ms
MFCC feature shape: 
(24, 27189)
(24, 27102)
(24, 25678)
(24, 25414)


In [5]:
feature_conca = np.zeros((feature[0].shape[0],1))
chick_id = []

for k in range(len(wav_files)):
    feature_conca = np.hstack((feature_conca, feature[k]))
    chick_id.extend([int(wav_files[k][:2])] * feature[k].shape[1])
        
chick_id = np.array(chick_id)
feature_conca = np.transpose(feature_conca)
feature_conca = feature_conca[1:]

In [6]:
# frame-wise label_id and call_id
frame_num = []
label_id = np.zeros((len(chick_id)), dtype=int)   # 1=pleasure, 2=distress, 3=unknown
call_id = np.zeros((len(chick_id)), dtype=int)

for chick in range(len(wav_files)):
    frame_num.append(feature[chick].shape[1])
    seg_chick = call_segs[call_segs['chickID']== int(wav_files[chick][:2])]
    sample_start = list(map(lambda x:int(x * sr / hop_sample) + sum(frame_num[:chick]), seg_chick['truth_start']))
    sample_end = list(map(lambda x:int(x * sr / hop_sample) + sum(frame_num[:chick]), seg_chick['truth_end']))
    
    for k in range(len(seg_chick)): # for each sample
        label_id[sample_start[k]:sample_end[k]] = np.zeros((sample_end[k]-sample_start[k])) + list(seg_chick['truth_label'])[k]
        call_id[sample_start[k]:sample_end[k]] = np.zeros((sample_end[k]-sample_start[k])) + list(seg_chick['callID'])[k]

In [7]:
print(feature_conca.shape, chick_id.shape, label_id.shape, call_id.shape)

(105383, 24) (105383,) (105383,) (105383,)


# Classification

In [8]:
# SVM settings
kernel = 'rbf'
gpu_id = 0 # thundersvm uses GPU to train SVM classifier
param_grid = {'C': [100,10,1], 'gamma': [.0001, .001, .01]}
scoring = 'f1_macro'
cv = 3

F_event = []; F_frame = []
dur = .5  # for event-based evaluation: duration of detected event should be at least 50% of the ground truth duration

In [9]:
# params for resampling predictions into the same frame size as that of the joint scattering feature
inframeSize = hop_sample / 44100 # ms
outframeSize = 0.186 # ms

subsample_rate = int(outframeSize / inframeSize)

## subject-independent train-test

In [10]:
for chick in chickID:
    
    subset = np.ones((len(label_id)), dtype=int) * 100

    for k in range(len(label_id)):
        if chick_id[k] !=chick and label_id[k] != 0:  
            subset[k] = 0
        elif chick_id[k] == chick and label_id[k] != 0:  # assign label for annotated (onset-offset) call segments
            subset[k] = 1

    feature_tr, label_tr = feature_conca[subset == 0], label_id[subset == 0]
    feature_te, label_te = feature_conca[subset == 1], label_id[subset == 1]

    # imputation
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    feature_tr = imp.fit_transform(feature_tr)
    feature_te = imp.transform(feature_te)

    # normalisation
    stdscaler = StandardScaler()
    feature_tr = stdscaler.fit_transform(feature_tr)
    feature_te = stdscaler.transform(feature_te)
    print(feature_tr.shape, feature_te.shape)

    # classification
    clf =  GridSearchCV(SVC(kernel=kernel, gpu_id=gpu_id), param_grid=param_grid, cv=cv, scoring=scoring)
    clf = clf.fit(feature_tr, label_tr)
    label_pred = clf.predict(feature_te)    
    
    ########## event-based evaluation ##########
    # majority vote of frame labels for each segment
    call_id_test = call_id[subset == 1]
    _, idx = np.unique(call_id_test, return_index=True)
    call_id_test_unique = call_id_test[np.sort(idx)]

    seg_label_te = []; seg_label_pred = []
    for call in call_id_test_unique:
        seg_label_te.append(collections.Counter(label_te[call_id_test==call]).most_common(1)[0][0])
        seg_label_pred.append(collections.Counter(label_pred[call_id_test==call]).most_common(1)[0][0])

    report = pd.DataFrame(classification_report(seg_label_te, seg_label_pred, output_dict=True))
    F_event.append([chick, report['1']['f1-score'], report['2']['f1-score'], report['3']['f1-score']])
    
    ########## frame-based evaluation ##########
    # test label back into frame level
    label_te_ori = label_id[chick_id==chick] # include label=0
    label_pred_true = np.zeros((len(label_te_ori)), dtype=int)
    label_pred_true[label_te_ori!=0] = label_pred

    label_te = label_te_ori
    label_pred = label_pred_true

    # resample into the same frame size as that of the JTFS feature
    label_te = label_te[::subsample_rate]
    label_pred = label_pred[::subsample_rate]
    
    report = pd.DataFrame(classification_report(label_te, label_pred, output_dict=True))
    F_frame.append([chick, report['1']['f1-score'], report['2']['f1-score'], report['3']['f1-score']])

(24802, 24) (4560, 24)
(20405, 24) (8957, 24)
(19087, 24) (10275, 24)
(23792, 24) (5570, 24)


## average

In [11]:
print('Frame-based F-scores for pleasure, contact, and uncertain calls: {}'.format(
    [round(elem, 2) for elem in np.mean(F_frame,0)[1:]*100]))
print('Event-based F-scores for pleasure, contact, and uncertain calls: {}'.format(
    [round(elem, 2) for elem in np.mean(F_event,0)[1:]*100]))

Frame-based F-scores for pleasure, contact, and uncertain calls: [16.77, 91.0, 8.46]
Event-based F-scores for pleasure, contact, and uncertain calls: [18.88, 87.31, 5.67]


In [12]:
print('Frame-based average F-score: {}'.format(round(np.mean(np.mean(F_frame,0)[1:]*100), 2)))
print('Event-based average F-score: {}'.format(round(np.mean(np.mean(F_event,0)[1:]*100), 2)))

Frame-based average F-score: 38.74
Event-based average F-score: 37.29
